<!-- # Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver (chromedriver / geckodriver) -->

In [66]:
!pip install selenium

In [67]:
# Como importar o selenium
# https://selenium-python.readthedocs.io/getting-started.html
from selenium import webdriver # permite controlar o navegador
from selenium.webdriver.common.keys import Keys # permite escrever no navegador
from selenium.webdriver.common.by import By # permite a seleção de itens no navegador

# para rodar o chrome em 2º plano - recomendável habilitação após concluir o código
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.headless = True 
navegador = webdriver.Chrome(options=chrome_options)

# Passo 1: Pegar a cotação do dólar
# abrir o navegador
navegador = webdriver.Chrome()
# caso queira deixar na mesma pasta do seu código
# navegador = webdriver.Chrome("chromedriver.exe")


# entrar no google
navegador.get("https://www.google.com.br/")

# pesquisar no google por "cotação dolar"
# utilizar sempre '' quando for passar o código do xpath
navegador.find_element("xpath",'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dolar")
navegador.find_element("xpath",'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
# pegar a cotação que tá no google
cotacao_dolar = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')


# Passo 2: Pegar a cotação do euro
# entrar no google
navegador.get("https://www.google.com.br/")
# pegar a cotação que tá no google
navegador.find_element("xpath",'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element("xpath",'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')


# Passo 3: Pegar a cotação do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")
cotacao_ouro = navegador.find_element('xpath','//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",",".")


print(cotacao_dolar)
print(cotacao_euro)
print(cotacao_ouro)


navegador.quit()

5.2013
5.122214233
288.43


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [68]:
# Passo 4: Atualizar a base de dados (preço de compra e venda)
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [69]:
# atualizar a coluna de cotação
# eu quero editar a coluna Cotação, onde a coluna Moeda == Dólar
tabela.loc[tabela["Moeda"]=="Dólar","Cotação"]= float(cotacao_dolar)
tabela.loc[tabela["Moeda"]=="Euro" ,"Cotação"]= float(cotacao_euro)
tabela.loc[tabela["Moeda"]=="Ouro" ,"Cotação"]= float(cotacao_ouro)

# atualizar a coluna de preço de compra = preço original * cotação
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# atualizar a coluna de preço de venda = preço compra * margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

#Para utlizar a coluna inteira = :
#Para formatar o campo número é = .map("R${:.2f}".format)
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.201300,5201.247987,1.40,7281.747182
1,Carro Renault,4500.00,Euro,5.122214,23049.964049,2.00,46099.928097
2,Notebook Dell,899.99,Dólar,5.201300,4681.117987,1.70,7957.900578
3,IPhone,799.00,Dólar,5.201300,4155.838700,1.70,7064.925790
4,Carro Fiat,3000.00,Euro,5.122214,15366.642699,1.90,29196.621128
5,Celular Xiaomi,480.48,Dólar,5.201300,2499.120624,2.00,4998.241248
6,Joia 20g,20.00,Ouro,288.430000,5768.600000,1.15,6633.890000


### Agora vamos exportar a nova base de preços atualizada

In [70]:
# Passo 5: Exportar a base de preços atualizada
tabela.to_excel("Produtos_Novo.xlsx", index=False)